In [1]:
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/workspace-git/flotilla/flotilla/__init__.py:55: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-02-10 09:53:22	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-02-10 09:53:22	Parsing datapackage to create a Study object
2016-02-10 09:54:06	Initializing Study
2016-02-10 09:54:06	Initializing Predictor configuration manager for Study
2016-02-10 09:54:06	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-02-10 09:54:06	Added ExtraTreesClassifier to default predictors
2016-02-10 09:54:06	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-02-10 09:54:06	Added ExtraTreesRegressor to default predictors
2016-02-10 09:54:06	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-02-10 09:54:06	Added GradientBoostingClassifier to default predictors
2016-02-10 09:54:06	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [2]:
folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian'
!mkdir $folder

figure_folder = '{}/kmer_counting'.format(folder)
! mkdir -p $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian': File exists


In [3]:
study.supplemental.modalities_tidy.groupby(['phenotype', 'modality']).size()

phenotype  modality  
MN         ambivalent       1
           bimodal        390
           excluded       566
           included      1639
NPC        ambivalent       2
           bimodal        799
           excluded      1092
           included      3047
iPSC       ambivalent      33
           bimodal       1383
           concurrent       1
           excluded      1605
           included      4611
dtype: int64

### Make BED files of each modality within each celltype

Use the other modalities, within that celltype, as background.

  - What properties are unique to events that are bimodal in iPSC, rather than ~0 or ~1?
    - Foreground: Bimodal events in iPSC
    - Background: All other events in iPSC

In [4]:


bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian'
! mkdir $bed_folder
import pybedtools

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian': File exists


In [5]:
import pyhomer

In [6]:
all_events = study.supplemental.modalities_tidy.event_id.unique()

In [7]:
from outrigger.region import Region

exon2s = map(lambda x: x.split('@')[1], all_events)
exon2_regions = map(Region, exon2s)
exon2_bed_table = pd.DataFrame(map(lambda x: [x.chrom, x._start, x._stop, x.name, 1000, x.strand], exon2_regions))
exon2_bed_table[3] = all_events
exon2_bed_table.head()

,0,1,2,3,4,5
0,chr10,102116289,102116521,exon:chr10:102114184-102114389:+@exon:chr10:10...,1000,+
1,chr10,102286156,102286311,exon:chr10:102286732-102286831:-@exon:chr10:10...,1000,-
2,chr10,103354415,103354495,exon:chr10:103348089-103348157:+@exon:chr10:10...,1000,+
3,chr10,103364897,103364969,exon:chr10:103360960-103361093:+@exon:chr10:10...,1000,+
4,chr10,103368592,103368694,exon:chr10:103360960-103361093:+@exon:chr10:10...,1000,+


In [8]:
exon2_bed_table = exon2_bed_table.sort_values(by=3)
exon2_bed_table.head()

,0,1,2,3,4,5
5163,chr10,101163481,101163631,exon:chr10:101165513-101165617:-@exon:chr10:10...,1000,-
5164,chr10,101478103,101478257,exon:chr10:101480744-101480825:-@exon:chr10:10...,1000,-
5165,chr10,101510126,101510153,exon:chr10:101507014-101507147:+@exon:chr10:10...,1000,+
5166,chr10,101996607,101996726,exon:chr10:101997779-101997988:-@exon:chr10:10...,1000,-
5167,chr10,101997779,101997988,exon:chr10:102003455-102003529:-@exon:chr10:10...,1000,-


In [9]:
exon2_bed_table.to_csv('{}/background_events.bed'.format(bed_folder), index=False, header=False, sep='\t')

In [10]:
from Bio import SeqIO
import kvector
import pybedtools

DIRECTIONS = 'upstream', 'downstream'



DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/background_events.bed'.format(bed_folder)
exon_bed = pybedtools.BedTool(exon_bedfile)

commands = []

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)


primate_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPrimates.bed'
primate = pybedtools.BedTool(primate_filename)
placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
placental = pybedtools.BedTool(placental_filename)
conserved_regions = {'primate': primate, 'placental': placental}

conservation_bed = placental

genome = 'hg19'

nt = 400


genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

kmer_zscores = []

for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    background_events = set(phenotype_df.event_id)
    for modality, modality_df in phenotype_df.groupby('modality'):
        print '---\n', phenotype, modality
        event_ids = set(modality_df.event_id)
        format_args = bed_folder, phenotype, modality
        foreground_table = exon2_bed_table.loc[exon2_bed_table[3].isin(event_ids)]
        foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
        foreground_table.to_csv(foreground_filename, index=False, header=False, sep='\t')
        foreground = pybedtools.BedTool(foreground_filename)
        
        
        background_table = exon2_bed_table.loc[exon2_bed_table[3].isin(background_events)]
        background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
        background_table.to_csv(background_filename, index=False, header=False, sep='\t')
        background = pybedtools.BedTool(background_filename)
        
        
        pair = pyhomer.ForegroundBackgroundPair(foreground, background)
#         print '\n', pair 
        
        for direction in DIRECTIONS:
            print '\n\t', direction
            intron_pair = pair.flanking_intron(direction, 'hg19', 400)

            conserved_introns = intron_pair.intersect(conservation_bed, 'placental')
#             print '\n', conserved_introns
            %time seqs = conserved_introns.foreground.sequence(fi=genome_fasta, s=True)
            %time foreground_kmers = kvector.count_kmers(seqs.seqfn)

            %time seqs = conserved_introns.background.sequence(fi=genome_fasta, s=True)
            %time background_kmers = kvector.count_kmers(seqs.seqfn)
            kmer_zscore = (foreground_kmers.mean() - background_kmers.mean())/background_kmers.std()
            kmer_zscore.name = '{}{}nt_{}_{}_placental'.format(direction, nt, phenotype, modality)
            kmer_zscores.append(kmer_zscore)
kmer_zscores_all = pd.concat(kmer_zscores, axis=1)
kmer_zscores_all.head()

---
MN ambivalent

	upstream
CPU times: user 2 ms, sys: 7 ms, total: 9 ms
Wall time: 32.2 ms
CPU times: user 399 ms, sys: 7 ms, total: 406 ms
Wall time: 801 ms
CPU times: user 2 ms, sys: 7 ms, total: 9 ms
Wall time: 3.41 s
CPU times: user 3min 6s, sys: 116 ms, total: 3min 6s
Wall time: 3min 6s

	downstream
CPU times: user 1e+03 µs, sys: 8 ms, total: 9 ms
Wall time: 23.5 ms
CPU times: user 439 ms, sys: 1e+03 µs, total: 440 ms
Wall time: 440 ms
CPU times: user 2 ms, sys: 7 ms, total: 9 ms
Wall time: 181 ms
CPU times: user 3min 3s, sys: 117 ms, total: 3min 3s
Wall time: 3min 4s
---
MN bimodal

	upstream
CPU times: user 2 ms, sys: 7 ms, total: 9 ms
Wall time: 33.9 ms
CPU times: user 42.9 s, sys: 26 ms, total: 43 s
Wall time: 43 s
CPU times: user 2 ms, sys: 8 ms, total: 10 ms
Wall time: 85.5 ms
CPU times: user 3min 7s, sys: 578 ms, total: 3min 7s
Wall time: 3min 8s

	downstream
CPU times: user 2 ms, sys: 8 ms, total: 10 ms
Wall time: 33.7 ms
CPU times: user 37.4 s, sys: 17 ms, total: 37.4 s

,upstream400nt_MN_ambivalent_placental,downstream400nt_MN_ambivalent_placental,upstream400nt_MN_bimodal_placental,downstream400nt_MN_bimodal_placental,upstream400nt_MN_excluded_placental,downstream400nt_MN_excluded_placental,upstream400nt_MN_included_placental,downstream400nt_MN_included_placental,upstream400nt_NPC_ambivalent_placental,downstream400nt_NPC_ambivalent_placental,...,upstream400nt_iPSC_ambivalent_placental,downstream400nt_iPSC_ambivalent_placental,upstream400nt_iPSC_bimodal_placental,downstream400nt_iPSC_bimodal_placental,upstream400nt_iPSC_concurrent_placental,downstream400nt_iPSC_concurrent_placental,upstream400nt_iPSC_excluded_placental,downstream400nt_iPSC_excluded_placental,upstream400nt_iPSC_included_placental,downstream400nt_iPSC_included_placental
AAAA,0.474524,-0.241359,0.185327,0.115022,0.168236,0.097491,-0.081924,-0.049461,0.475049,-0.247089,...,0.257730,0.473247,0.084969,0.016316,0.264650,-0.244265,0.182092,0.159253,-0.073529,-0.049269
AAAC,-0.247947,-0.244669,0.147088,0.040678,0.186828,0.152138,-0.076310,-0.044002,-0.244190,-0.242973,...,0.301763,0.098104,0.046665,0.003262,-0.237077,-0.234942,0.155938,0.168336,-0.055657,-0.043485
AAAG,0.265535,-0.067158,0.100481,0.041271,0.192364,0.123487,-0.067273,-0.038083,0.807236,-0.296720,...,0.459921,0.225355,0.028947,-0.001096,0.502446,-0.288491,0.180525,0.155642,-0.058116,-0.040434
AAAT,-0.301619,-0.304202,0.201983,0.130975,0.271184,0.145468,-0.108141,-0.064133,0.636355,-0.299703,...,0.331626,0.550518,0.119045,0.031195,0.340351,-0.294706,0.218485,0.194627,-0.093558,-0.063249
AACA,-0.285755,-0.257514,0.109675,0.006757,0.134356,0.180587,-0.055661,-0.042285,-0.272352,-0.256995,...,0.405794,0.293585,0.041910,-0.026416,-0.257180,-0.245345,0.108622,0.187163,-0.043230,-0.041479


In [11]:
kmer_zscores_all = pd.concat(kmer_zscores, axis=1)
print kmer_zscores_all.shape
kmer_zscores_all = kmer_zscores_all.dropna(axis=1, how='all').dropna(how='all', axis=0)
print kmer_zscores_all.shape

# Replace remaining NAs with zero since they aren't enriched
kmer_zscores_all = kmer_zscores_all.fillna(0)
kmer_zscores_all.head()

(5376, 26)
(5376, 26)


,upstream400nt_MN_ambivalent_placental,downstream400nt_MN_ambivalent_placental,upstream400nt_MN_bimodal_placental,downstream400nt_MN_bimodal_placental,upstream400nt_MN_excluded_placental,downstream400nt_MN_excluded_placental,upstream400nt_MN_included_placental,downstream400nt_MN_included_placental,upstream400nt_NPC_ambivalent_placental,downstream400nt_NPC_ambivalent_placental,...,upstream400nt_iPSC_ambivalent_placental,downstream400nt_iPSC_ambivalent_placental,upstream400nt_iPSC_bimodal_placental,downstream400nt_iPSC_bimodal_placental,upstream400nt_iPSC_concurrent_placental,downstream400nt_iPSC_concurrent_placental,upstream400nt_iPSC_excluded_placental,downstream400nt_iPSC_excluded_placental,upstream400nt_iPSC_included_placental,downstream400nt_iPSC_included_placental
AAAA,0.474524,-0.241359,0.185327,0.115022,0.168236,0.097491,-0.081924,-0.049461,0.475049,-0.247089,...,0.257730,0.473247,0.084969,0.016316,0.264650,-0.244265,0.182092,0.159253,-0.073529,-0.049269
AAAC,-0.247947,-0.244669,0.147088,0.040678,0.186828,0.152138,-0.076310,-0.044002,-0.244190,-0.242973,...,0.301763,0.098104,0.046665,0.003262,-0.237077,-0.234942,0.155938,0.168336,-0.055657,-0.043485
AAAG,0.265535,-0.067158,0.100481,0.041271,0.192364,0.123487,-0.067273,-0.038083,0.807236,-0.296720,...,0.459921,0.225355,0.028947,-0.001096,0.502446,-0.288491,0.180525,0.155642,-0.058116,-0.040434
AAAT,-0.301619,-0.304202,0.201983,0.130975,0.271184,0.145468,-0.108141,-0.064133,0.636355,-0.299703,...,0.331626,0.550518,0.119045,0.031195,0.340351,-0.294706,0.218485,0.194627,-0.093558,-0.063249
AACA,-0.285755,-0.257514,0.109675,0.006757,0.134356,0.180587,-0.055661,-0.042285,-0.272352,-0.256995,...,0.405794,0.293585,0.041910,-0.026416,-0.257180,-0.245345,0.108622,0.187163,-0.043230,-0.041479


In [12]:
study.supplemental.kmer_zscores = kmer_zscores_all

In [13]:
kmer_zscores_metadata = pd.DataFrame.from_records(
    study.supplemental.kmer_zscores.columns.map(lambda x: pd.Series(x.split('_'))), 
    index=study.supplemental.kmer_zscores.columns)
kmer_zscores_metadata.columns = ['direction', 'phenotype', 'modality', 'clade']
study.supplemental.kmer_zscores_metadata = kmer_zscores_metadata

kmer_zscores_metadata.head()

,direction,phenotype,modality,clade
upstream400nt_MN_ambivalent_placental,upstream400nt,MN,ambivalent,placental
downstream400nt_MN_ambivalent_placental,downstream400nt,MN,ambivalent,placental
upstream400nt_MN_bimodal_placental,upstream400nt,MN,bimodal,placental
downstream400nt_MN_bimodal_placental,downstream400nt,MN,bimodal,placental
upstream400nt_MN_excluded_placental,upstream400nt,MN,excluded,placental


In [14]:
study.save('singlecell_pnm_figure2_modalities_bayesian_kmers', flotilla_dir=flotilla_dir)

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian_kmers/datapackage.json
